# End to end reconstruction 

In [1]:
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tables import open_file

In [2]:
import sys
sys.path.insert(0,'../utils')

In [3]:
from gammalearn import load_model

In [4]:
import warnings
warnings.filterwarnings(action='ignore')

In [5]:
exp_path = '/Users/thomasvuillaume/Work/GammaLearn/experiments/'
# exp_name = 'R_0266'
# checkpoint = '100'
exp_name = 'R_0300'
checkpoint = '100'
camera_model_path = '/Users/thomasvuillaume/Work/GammaLearn/GammaLearn/share/camera_parameters.h5'

In [6]:
model = load_model(exp_path, exp_name, checkpoint, camera_model_path)

n_pixels: 29


In [7]:
softmax = torch.nn.Softmax()

In [8]:
dl1_filename = '../dl1_data/dl1_gamma_test_large.simtel.h5'

In [9]:
dl1_table = pd.read_hdf(dl1_filename, key='events/LSTCam')
dl1_table.head()

,disp_angle,disp_dx,disp_dy,disp_miss,disp_norm,disp_sign,event_id,gps_time,intensity,intercept,...,src_y,tel_id,tel_pos_x,tel_pos_y,tel_pos_z,time_gradient,width,wl,x,y
0,1.213061,0.283341,0.757963,0.088727,0.809191,1.0,31012,1.467925e+09,2.726635,10.772820,...,1.142787,2,-20.0,-65.0,16.0,-4.199312,0.099501,0.203462,-0.966779,0.384824
1,-1.315711,-0.109912,0.421497,0.006006,0.435592,-1.0,31012,1.467925e+09,4.386854,12.429269,...,1.142787,3,80.0,0.0,16.0,1.257596,0.174557,0.393136,-0.573526,0.721290
2,0.752671,-0.226153,-0.211814,0.083606,0.309856,-1.0,90914,1.467925e+09,1.950573,9.307944,...,0.502322,2,-20.0,-65.0,16.0,1.867749,0.039196,0.408410,0.569424,0.714137
3,-1.566774,0.001596,-0.396883,0.054431,0.396886,1.0,90914,1.467925e+09,2.044103,8.797758,...,0.502322,4,-120.0,0.0,16.0,-0.163486,0.049995,0.488243,0.341674,0.899205
4,1.462424,-0.041423,-0.380728,0.130275,0.382975,-1.0,153614,1.467925e+09,1.903458,8.764436,...,0.467924,1,-20.0,65.0,16.0,-4.672784,0.048217,0.495253,-0.461488,0.848652


In [10]:
with open_file(dl1_filename) as file:
    images = file.root.events.LSTCam_images[:]['image']
    pulse_times = file.root.events.LSTCam_images[:]['pulse_time']

In [11]:
images.shape, dl1_table.shape

((69, 1855), (69, 41))

In [12]:
data = torch.tensor(np.transpose([images, pulse_times], axes=[1,0,2]), dtype=torch.float)

In [13]:
prediction = model(data).squeeze(0).detach().numpy()

In [14]:
softmax = torch.nn.Softmax(dim=1)
soft = softmax(torch.tensor(prediction[:, 5:]))